In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
# import things
import tensorflow as tf
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from keras.applications.vgg16 import VGG16

In [3]:
# reading dataset files
import time
from tqdm import tqdm
# unpacking data
from zipfile import ZipFile
with ZipFile("drive/My Drive/mnist/Medical MNIST.zip","r") as zip_ref:
     for file in tqdm(iterable=zip_ref.namelist(), total=len(zip_ref.namelist())):
          zip_ref.extract(member=file,path="mnist")
# put dataset path in below variable
directory = "./mnist"
# # Medical Mnist Image size is 64 * 64
data = tf.keras.preprocessing.image_dataset_from_directory(
    directory, color_mode='rgb', batch_size=32, image_size=(64, 64), shuffle=True
)
# # reading vgg16 model
model = VGG16(weights='imagenet' , include_top=False , input_shape=(64,64,3))
# # freezing layers to add new layers later
for layer in model.layers:
  layer.trainable = False

100%|██████████| 58954/58954 [00:13<00:00, 4358.89it/s]


Found 58954 files belonging to 6 classes.


In [5]:
# changing model
output = model.output
layer = tf.keras.layers.Flatten()(output)
layer = tf.keras.layers.Dense(units=2048)(layer)
layer = tf.keras.layers.Dropout(0.05)(layer) # avoid overfitting
layer = tf.keras.layers.Dense(units=2048)(layer)
layer = tf.keras.layers.Dense(units=6 , activation='softmax')(layer)
model = tf.keras.Model(
        inputs=model.input,
        outputs=layer,
    )
# compiling model
# sparse categorical loss function means labels are integers but output is 6 length tensor
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , optimizer="Adam" , metrics = ["accuracy"])
print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0   

In [7]:
# train model using new data
train = data.take(int(len(data)*70/100))
test = data.skip(int(len(data)*70/100)).take(int(len(data)*30/100))
model.fit(train , epochs=1 ,batch_size=32)

1843/1843 [==============================] - 2382s 1s/step - loss: 1.7930 - accuracy: 0.1714


In [26]:
model.evaluate(test)

552/552 [==============================] - 697s 1s/step - loss: 1.7936 - accuracy: 0.1680


[1.7936094999313354, 0.16796875]